In [ ]:
import pandas as pd
from IPython.display import display
from pandasgui import show

- https://pandas.pydata.org/docs/reference/api/pandas.read_excel.html
- https://pandas.pydata.org/docs/reference/api/pandas.option_context.html
- http://geoportal.cm-amadora.pt/graffiti/Arte_Urbana/index.html

In [ ]:
RAW_DATA = "./arte_urbana.xlsx"
OUTPUT_DATA = "../src/data.json"

In [ ]:
df = pd.read_excel(RAW_DATA, index_col="ID", verbose=True, dtype={"Data": str})

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.shape

In [ ]:
df.dtypes

In [ ]:
df.isna().sum()

In [ ]:
with pd.option_context("display.max_rows", None):
    display(df["Autoria"].value_counts(dropna=False))

In [ ]:
with pd.option_context("display.max_rows", None):
    display(df["Data"].value_counts(dropna=False))

In [ ]:
df.loc[df["Data"].isna(), "Descricao"].to_list()

## Data Processing

In [ ]:
# df["Autoria"] = df["Autoria"].str.replace("artistas:", "", regex=False)
# df["Autoria"] = df["Autoria"].str.replace("\s/\s", "/", regex=True)

## Output

In [ ]:
indent = 2

df.to_json(OUTPUT_DATA, orient="records", force_ascii=False, indent=indent)

---